In [1]:
import sys
sys.path.append('../src')
import tarski
import tarski.fstrips as fs
import tarski.syntax.temporal.ltl as ltl
from tarski.symbols import *

# Planning with Reactions is Checking for $\phi U \varphi$ Specifications

We will start this notebook borrowing the discussion in Chapter 3, Section 3.5 of [Huth & Ryan's _Logic in Computer Science_](https://www.amazon.com/Logic-Computer-Science-Modelling-Reasoning/dp/052154310X)

## The ferryman

You may recall the puzzle of a ferryman, goat, cabbage, and wolf all on one side of a river. The ferryman can cross the river with **at most** one passenger in his boat. There is behavioural conflict between:

 1. The goat and the cabbage; and
 2. the goat and the wolf;
 
if they are on the same river bank but the ferryman crosses the river or stays on the other bank.

Can the ferryman transport all goods to the other side, without any conflicts occurring? This is a _planning problem_, but it can be solved by model checking. We describe a transition system in which the **states represent which goods are at which side of the river**. Then we ask if the goal state is reachable from the initial state: Is there a path from the initial state such that it has a state along it at which all the goods are on the other side, and during the transitions to that state the goods are never left in an _unsafe_, _conflicting_ situation?

We model all possible behaviour (including that which results in conflicts) as a ```NuSMV``` program:

```
MODULE main
    VAR
        ferryman: {0,1};
        goat: {0,1};
        cabbage: {0,1};
        wolf: {0,1};
        carry: {g,c,w,0};
    ASSIGN
        init(ferryman) := 0;
        init(goat) := 0;
        init(cabbage) := 0;
        init(wolf) := 0;
        init(carry) := 0;

        next(ferryman) := {0,1};

        next(carry) := case
                        ferryman=goat : g;
                        TRUE :0;
                       esac union
                       case
                        ferryman=cabbage : c;
                        TRUE :0;
                       esac union
                       case
                        ferryman=wolf : w;
                        TRUE : 0;
                       esac union 0;
        next(goat) := case
            ferryman=goat & next(carry) = g : next(ferryman);
            TRUE : goat;
        esac;
        next(cabbage) := case
            ferryman=cabbage & next(carry) = c : next(ferryman);
            TRUE : cabbage;
        esac;
        next(wolf) := case
            ferryman=wolf & next(carry)=w : next(ferryman);
            TRUE : wolf;
        esac;
    LTLSPEC !((  (goat=cabbage | goat=wolf) -> goat=ferryman) U (cabbage=1 & goat=1 & wolf=1 & ferryman=1))
```

The _location_ of each agent is modelled as a Boolean variable: $0$ denotes that agent is on the initial bank, and 1 the destination bank. Thus, ```ferryman=0``` means that the ferryman is on the _initial_ bank, ```ferryman=1``` that he is on the destination bank, and similarly for the variables ```goat```, ```cabbage``` and ```wolf```.

The variable ```carry``` takes a value indicating whether the goat, cabbage, wolf or _nothing_ is carried by the ferryman. The definition of ```next(carry)``` works as follows. It is non-deterministic, but the set from which a value is non--deterministically chosen is determined by the values of ferryman, goat, etc. and **always** includes ```0```. If ```ferryman=goat```, i.e. they are on the same side, then ```g``` is a member of the set which ```next(carry)``` is chosen. The situation for cabbage and wolf is similar. Thus, if

$$
ferryman = goat = wolf \neq cabbage
$$

then that set is 

$$
\{ g, w, 0 \}
$$

The next value assigned to ```ferryman``` is non-deterministic: he can _choose_ to cross or not to cross the river. But the next values of ```goat```, ```cabbage``` and ```wolf``` are _deterministic_, since whether they are carried or not is determined by the ferryman's choice, represented by the non-deterministic assignment to ```carry```; these values follow the same pattern.

Note how the boolean guards for ```next(goat)```, ```next(cabbage)``` and ```next(wolf)``` all refer to state bits _at the next state_. The SMV compiler does a _dependency analysis_ and rejects circular dependencies on next values. _NB: The dependency analysis is rather pessimistic, sometimes NuSMV complains of circularity even in situations when it could be resolved. The original CMU-SMV is more liberal in this respect._

### Running SMV

We seek a path satisfying $\phi U \psi$, where $\psi$ asserts the **final goal state**, and $\phi$ expresses the **safety condition**. Namely, if the goat is with the cabbage or the worlf, then the ferryman is there too, to prevent anything untoward happening. Thus, we assert that _all paths_ satisfy $\neg(\phi U \psi)$, i.e. **no path** satisfies $\phi U \psi$. We **hope** this is not the case, and NuSMV will give us an example which _does_ satisfy $\phi U \psi$.

The beginning of the generated path represents the usual solution to this
puzzle: the ferryman takes the goat first, then goes back for the cabbage. To
avoid leaving the goat and the cabbage together, he takes the goat back, and
picks up the wolf. Now the wolf and the cabbage are on the destination side,
and he goes back again to get the goat. This brings us to State 1.9, where
the ferryman appears to take a well-earned break. But the path continues.
States 1.10 to 1.15 show that he takes his charges back to the original side
of the bank; first the cabbage, then the wolf, then the goat. Unfortunately
it appears that the ferryman’s clever plan up to state 1.9 is now spoiled,
because the goat meets an unhappy end in state 1.11.

In [2]:
!NuSMV ferryman.smv

*** This is NuSMV 2.6.0 (compiled on Wed Oct 14 15:35:00 2015)
*** Enabled addons are: compass
*** For more information on NuSMV see <http://nusmv.fbk.eu>
*** or email to <nusmv-users@list.fbk.eu>.
*** Please report bugs to <Please report bugs to <nusmv-users@fbk.eu>>

*** Copyright (c) 2010-2014, Fondazione Bruno Kessler

*** This version of NuSMV is linked to the CUDD library version 2.4.1
*** Copyright (c) 1995-2004, Regents of the University of Colorado

*** This version of NuSMV is linked to the MiniSat SAT solver. 
*** See http://minisat.se/MiniSat.html
*** Copyright (c) 2003-2006, Niklas Een, Niklas Sorensson
*** Copyright (c) 2007-2010, Niklas Sorensson

WARNING *** This version of NuSMV is linked to the zchaff SAT         ***
WARNING *** solver (see http://www.princeton.edu/~chaff/zchaff.html). ***
WARNING *** Zchaff is used in Bounded Model Checking when the         ***
WARNING *** system variable "sat_solver" is set to "zchaff".          ***
WARNING *** Notice that zchaff is

What went wrong? Nothing, actually. NuSMV has given us an infinite
path, which loops around the 15 illustrated states. Along the infinite path,
the ferryman repeatedly takes his goods across (safely), and then back again
(unsafely). This path does indeed satisfy the specification $\phi U \psi$, which asserts
the safety of the forward journey but says nothing about what happens
after that. In other words, the path is correct; it satisfies $\phi U \psi$ (with $\psi$ occurring
at state 8). What happens along the path after that has no bearing
on $\phi U \psi$.

Invoking bounded model checking will produce the shortest possible path
to violate the property; in this case, it is states 1.1 to 1.8 of the illustrated
path. It is the shortest, optimal solution to our planning problem
since the model check 

```
NuSMV -bmc -bmc_length 7 ferryman.smv
``` 

shows that the LTL formula holds in that model, meaning that no solution with
fewer than seven transitions is possible.

One might wish to verify whether there is a solution which involves three
journeys for the goat. This can be done by altering the LTL formula. Instead
of seeking a path satisfying $\phi U \psi$, where

$$
\phi \equiv (goat=cabbage \lor goat=wolf) \rightarrow (goat=ferryman)
$$

and

$$
\psi \equiv cabbage \land goat \land wolf \land ferryman
$$

we now seek a path satisfying 

$$
(\phi U \psi) \land G(goat \rightarrow G\,goat)
$$

The last bit says that once the goat has crossed, he remains across; otherwise, the goat makes at least three trips. NuSMV verifies that the negation of this formula is true, confirming that there is no such solution.

## Accounting for the Semantics of the ```ferryman``` NuSMV program 

** TODO **: Research Questions:
 - Find works studying whether $PSPACE$ is closed under the complement or not.

### Variables

In [3]:
f = tarski.language('ferryman')

We create constants of sort ```object``` for each of the entities in the puzzle

In [4]:
cargo = f.sort('cargo')
e = tarski.Bunch(ferryman=f.constant('ferryman',f.Object),\
                goat=f.constant('goat',cargo),\
                cabbage=f.constant('cabbage',cargo),\
                wolf=f.constant('wolf', cargo))

the location of each of these entities, represented with the variables ```ferryman```, etc. can be accounted for having a simple function to denote the location of each object

In [5]:
location = f.function('location',f.Object, f.Integer)

The set ```carry``` can be accounted for introducing a _predicate_ which implements the [characteristic function](https://en.wikipedia.org/wiki/Indicator_function) for the set in question, so that whenever an object ```o``` is in the set ```carry```, the predicate needs to hold

In [6]:
carrying = f.predicate('carrying', cargo)

i.e. whenever $carrying(goat)$ holds, then $goat \in carry$.

**NOTE**: sets are essentially then syntactic sugar, we could have in the language the sort ```set of [Sort]```, and each object be the name of a singular set. Then we can introduce automatically predicates $in(x,S)$ where $x$ that we set to true and false _automatically_ as we come across operators such as $union(S,S')$, $union(S,x)$, $minus(S,S')$, $minus(S,x)$,...

### Non-Deterministic Transitions as Actions

#### ```next(ferryman)```

As per the specification of Huth & Ryan, changes in value of the location of the ferryman are non-deterministic as in chosen by the _environment_. In this particular this role is played by the planner itself: the trajectories satisfying the specification will be changing the value of ```ferryman``` as the planner chooses a particular action to be executed.

We introduce actions

In [7]:
go_forth = fs.Action(f, name='go_forth', parameters=[], precondition=top, effects=[ltl.X(location(e.ferryman)==1)])

In [8]:
go_back = fs.Action(f, name='go_back', parameters=[], precondition=top, effects=[ltl.X(location(e.ferryman)==0)])

It is worth noting that we could achieve a more efficient encoding forbidding the transitions $0 \rightarrow 0$ and $1 \rightarrow 1$.

#### ```next(carry)```

Similarly to the case of the transition above, the planner is also expected to choose amongst any of the following changes to happen

In [9]:
c = f.Variable('c', cargo)
pick = fs.Action(f, name='pick', parameters=[c], \
                 precondition=[location(e.ferryman)==location(c)],\
                 effects=[ ltl.X(carrying(c)) ])

### Deterministic Transitions as Reactions

The rest of the entries in the NuSMV program transition deal with the _non voluntary_ or _natural_ consequences of the choices made by the planner/environment. We specify these as constraints:

In [10]:
x = f.Variable('x',cargo)
R = forall( x, \
           implies( (location(e.ferryman) == location(x)) & ltl.X(carrying(x)),\
                      ltl.X(location(x) == location(e.ferryman))))

### LTL Specification

The safety constraint becomes just another constraint

In [11]:
S = implies(lor(location(e.goat) == location(e.cabbage), location(e.goat)==location(e.wolf)), \
            location(e.goat)==location(e.ferryman))

and the goal is simply

In [12]:
o = f.Variable('o',f.Object)
G = forall(o, location(o)==1)

### Putting it all together

**NOTE**: parallel action support need to be set

In [13]:
P = fs.Problem( "ferryman", f, init=None, goal=G, constraints=[R, S], actions=[go_forth,go_back,pick])

IncompleteProblemError: Problem "ferryman": No initial state was given